In [1]:
import os
import math
import itertools
import numpy as np
import pandas as pd
# PyTorch
import torch
# Matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [2]:
import sys
sys.path.append('../src/')
%load_ext autoreload
%autoreload 2
# Importing our custom module(s)
import metrics
import models
import priors
import utils

In [3]:
repo_path = '/cluster/home/eharve06/extrapolating-classifier-accuracy-to-bigger-datasets'
experiments_path = os.path.join(repo_path, 'experiments')
models_path = os.path.join(repo_path, 'models')
models_without_priors_path = os.path.join(repo_path, 'models_without_priors')

In [4]:
def bootstrap(row, size):
    if int(row.name)<=360:
        return np.mean(row.test_auroc, axis=0)
    else:
        num_splits, num_labels = np.array(row.test_auroc).shape        
        samples = [np.random.choice(np.array(row.test_auroc)[:,label_index], size=size, replace=True) for label_index in range(num_labels)]
        return np.mean(samples, axis=1)

def coverage_rates(experiments_path, dataset_name, distance, label_name, models_path, model_name, size):
    # Get label index
    label_index = None
    labels = [['Atelectasis', 'Effusion', 'Infiltration'], ['Bacterial', 'Viral'], 
              ['Normal', 'Benign', 'Malignant'], ['PLAX', 'PSAX', 'A4C', 'A2C'], 
              ['Alzheimer’s'], ['WMD', 'CBI']]
    for label in labels:
        if label_name in label:
            label_index = label.index(label_name)
    assert label_index is not None, 'label not found in labels_list'
    
    filename = '{}_{}.csv'.format(dataset_name, distance)
    df = utils.load_experiment(os.path.join(experiments_path, filename))
    # Take mean of each random seed at each dataset size
    df = df.groupby('n').agg(lambda x: list(x))
    df.test_auroc = df.test_auroc.apply(lambda x: np.mean(x, axis=0))
    df.random_state = df.random_state.apply(lambda x: 'mean')
    df = df.reset_index()
    
    X_train, y_train, _, _ = utils.split_df(df, index=label_index)
    model_filename = '{}_{}_{}.pt'.format(dataset_name, label_name, model_name)
    print(model_filename)
    model_filepath = os.path.join(models_path, model_filename)
    model_objects = utils.load_model(model_name, model_filepath, X_train, y_train)
    
    # TODO: Start loop
    tpms = []
    for i in range(500):
        df = utils.load_experiment(os.path.join(experiments_path, filename))
        # Take mean of each random seed at each dataset size
        df = df.groupby('n').agg(lambda item: list(item))
        df.test_auroc = df.apply(lambda item: bootstrap(item, size), axis=1)
        df.random_state = df.random_state.apply(lambda x: 'mean')
        df = df.reset_index()

        _, _, X_test, y_test = utils.split_df(df, index=label_index)
        model, *likelihood_objects = model_objects
        likelihood, = likelihood_objects
        with torch.no_grad(): predictions = likelihood(model(X_test))
        loc = predictions.mean.numpy()
        scale = predictions.stddev.numpy()
        tpm = priors.truncnorm_probability_mass(y_test.detach().numpy(), 0.0, 1.0, loc, scale)
        tpms.append(tpm)
    #print('${:.1f}\pm{:.1f}\%$ &'.format(100*np.mean(tpms), 100*np.std(tpms)))
    print('${:.1f}\% ({:.1f}-{:.1f})$ &'.format(100*np.mean(tpms), 100*np.min(tpms), 100*np.max(tpms)))

In [5]:
coverage_rates(experiments_path, 'ChestX-ray14', 'short_range', 'Atelectasis', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'short_range', 'Atelectasis', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'long_range', 'Atelectasis', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'long_range', 'Atelectasis', models_path, 'GPArctan', 3)

ChestX-ray14_Atelectasis_GPPowerLaw.pt
$45.1\% (30.1-54.6)$ &
ChestX-ray14_Atelectasis_GPArctan.pt
$44.5\% (26.7-53.7)$ &
ChestX-ray14_Atelectasis_GPPowerLaw.pt
$29.4\% (19.1-35.3)$ &
ChestX-ray14_Atelectasis_GPArctan.pt
$21.9\% (13.5-27.9)$ &


In [6]:
coverage_rates(experiments_path, 'ChestX-ray14', 'short_range', 'Effusion', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'short_range', 'Effusion', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'long_range', 'Effusion', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'long_range', 'Effusion', models_path, 'GPArctan', 3)

ChestX-ray14_Effusion_GPPowerLaw.pt
$38.1\% (24.5-47.2)$ &
ChestX-ray14_Effusion_GPArctan.pt
$38.0\% (24.9-46.8)$ &
ChestX-ray14_Effusion_GPPowerLaw.pt
$15.2\% (8.8-19.4)$ &
ChestX-ray14_Effusion_GPArctan.pt
$15.1\% (9.0-19.7)$ &


In [7]:
coverage_rates(experiments_path, 'ChestX-ray14', 'short_range', 'Infiltration', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'short_range', 'Infiltration', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'long_range', 'Infiltration', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'ChestX-ray14', 'long_range', 'Infiltration', models_path, 'GPArctan', 3)

ChestX-ray14_Infiltration_GPPowerLaw.pt
$44.9\% (33.2-51.9)$ &
ChestX-ray14_Infiltration_GPArctan.pt
$23.6\% (9.2-42.5)$ &
ChestX-ray14_Infiltration_GPPowerLaw.pt
$25.5\% (16.7-30.7)$ &
ChestX-ray14_Infiltration_GPArctan.pt
$1.0\% (0.0-6.3)$ &


In [8]:
coverage_rates(experiments_path, 'Chest_X-Ray', 'short_range', 'Bacterial', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'Chest_X-Ray', 'short_range', 'Bacterial', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'Chest_X-Ray', 'long_range', 'Bacterial', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'Chest_X-Ray', 'long_range', 'Bacterial', models_path, 'GPArctan', 3)

Chest_X-Ray_Bacterial_GPPowerLaw.pt
$42.4\% (16.8-61.0)$ &
Chest_X-Ray_Bacterial_GPArctan.pt
$42.7\% (17.5-61.0)$ &
Chest_X-Ray_Bacterial_GPPowerLaw.pt
$37.0\% (4.4-58.5)$ &
Chest_X-Ray_Bacterial_GPArctan.pt
$44.0\% (18.9-59.0)$ &


In [ ]:
coverage_rates(experiments_path, 'Chest_X-Ray', 'short_range', 'Viral', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'Chest_X-Ray', 'short_range', 'Viral', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'Chest_X-Ray', 'long_range', 'Viral', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'Chest_X-Ray', 'long_range', 'Viral', models_path, 'GPArctan', 3)

Chest_X-Ray_Viral_GPPowerLaw.pt
$39.7\% (20.2-55.4)$ &
Chest_X-Ray_Viral_GPArctan.pt
$38.7\% (18.6-53.5)$ &
Chest_X-Ray_Viral_GPPowerLaw.pt


In [ ]:
coverage_rates(experiments_path, 'BUSI', 'short_range', 'Normal', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'BUSI', 'short_range', 'Normal', models_path, 'GPArctan', 3)

In [ ]:
coverage_rates(experiments_path, 'BUSI', 'short_range', 'Benign', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'BUSI', 'short_range', 'Benign', models_path, 'GPArctan', 3)

In [ ]:
coverage_rates(experiments_path, 'BUSI', 'short_range', 'Malignant', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'BUSI', 'short_range', 'Malignant', models_path, 'GPArctan', 3)

In [ ]:
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'PLAX', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'PLAX', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'PLAX', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'PLAX', models_path, 'GPArctan', 3)

In [ ]:
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'PSAX', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'PSAX', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'PSAX', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'PSAX', models_path, 'GPArctan', 3)

In [ ]:
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'A4C', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'A4C', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'A4C', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'A4C', models_path, 'GPArctan', 3)

In [ ]:
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'A2C', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'short_range', 'A2C', models_path, 'GPArctan', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'A2C', models_path, 'GPPowerLaw', 3)
coverage_rates(experiments_path, 'TMED-2', 'long_range', 'A2C', models_path, 'GPArctan', 3)

In [ ]:
coverage_rates(experiments_path, 'OASIS-3', 'short_range', 'Alzheimer’s', models_path, 'GPPowerLaw', 15)
coverage_rates(experiments_path, 'OASIS-3', 'short_range', 'Alzheimer’s', models_path, 'GPArctan', 15)

In [ ]:
coverage_rates(experiments_path, 'Pilot', 'short_range', 'WMD', models_path, 'GPPowerLaw', 15)
coverage_rates(experiments_path, 'Pilot', 'short_range', 'WMD', models_path, 'GPArctan', 15)

In [ ]:
coverage_rates(experiments_path, 'Pilot', 'short_range', 'CBI', models_path, 'GPPowerLaw', 15)
coverage_rates(experiments_path, 'Pilot', 'short_range', 'CBI', models_path, 'GPArctan', 15)